# Genetic Altorithm 구현하기 

#### Genetic Algorithm 과정

1. 염색체(Chromesome) 초기화 및 파라미터 설정
  
  - 염색체 초기화 : 각 염색체의 변수마다 랜덤하게 사용 유무 결정
    
  - 파라미터 : 염색체 수 / 성능 평가 방식(Fitness fuction) / 교배방식(Crossover mechanism) / 돌연변이율(Rate of mutation) / 종료 조건
    
2. 각 염색체 선택 변수별 모델 학습
  
3. 각 염색체 적합도 평가
  
4. 우수 염색체 선택(Selection)
  
  - 결정론적(Deterministic) 선택 : 성능이 좋은 상위 n%의 염색체만 선정
    
  - 확률존적(Probabilistic) 선택 : 성능에 비례하여 각 염색체가 선정될 확률 부여.
    
5. 다음 세대 염색체 생성(Crossoveer & Mutation)
  
  - Crossover : 선택한 염색체들 간 변수값을 교체함.
    
  - Mutation : 일정 확률로 변수의 값을 변경함
   
  > 이전 세대의 최고 성능을 가진 염색체는 그대로 물려주며 원하는 성능이 나오기 까지 2~5번 과정 반복

  
6. 최종 변수 집합 선택

In [4]:
# 데이터 사용 및 라이브러리 설치 

import numpy as np
import pandas as pd
import random

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression 

boston = load_boston()
X = boston.data 
y = boston.target
model = LinearRegression()

### 1. 염색체 초기화 및 파라미터 설정 (__init__)  ~ 3. 적합도 평가

**구현해야하는 것** 
- fitness function : 설명력 지표 / 평가 수단
> adj_R_sq / weight 필요 
- chromosome 생성 : 각 변수값마다 뽑히냐 마냐(0 or 1)를 결정하는 list 값. 

**필요한 것**
- X : 입력 데이터  
- y : target 데이터 
- pred_y : 예상 결과값 
- c : 염색체의 수 
- m : 변수의 수 
- way : 어떻게 교배할 것인지. (Deterministic / Probability) 
- r : 돌연변이율 
- purpose : 종료조건 => 마지막 "최종 변수 집합" 단계에서 부여  

**함수의 형태**
- __init__ 함수를 통해서 파라미터 설정 
- Chromosome 생성 함수 설정 
- adj_R_sq / weight 각각의 개별 함수 설정. 


In [31]:
class Genetic_Algorithm() :
    def __init__(self, X, y, model, way, c, r, c_ratio) :
        self.X = X
        self.y = y
        self.model = model
        self.way = way
        self.c = c
        self.r = r 
        self.c_ratio = c_ratio
        
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]

        self.all_vars = np.array(range(self.n))
        self.chromosome = self.create_chrom()

    def create_chrom(self) : 
        chromosome = []
        for i in range(self.c) : 
            chrom = np.random.choice([0, 1], size=(self.m), p=[1-self.c_ratio, self.c_ratio]) 
            chromosome.append(chrom)
        return np.array(chromosome)
            
        
    def ad_R_sq(self, chrom) : 
        # chrom : 단일 염색체. np.array의 형식으로 0,1의 값으로 이뤄져 있음.
        select_val = [c for c, v in zip(self.all_vars, chrom) if v == 1]
        k = len(select_val)
        select_X = np.take(self.X, select_val, axis=1)
        mean_y = np.mean(self.y)

        self.model.fit(select_X,self.y)
        pred_y = self.model.predict(select_X)

        SST = np.sum((self.y - mean_y)**2)
        SSE = np.sum((self.y-pred_y)**2) 
    
        ad_R_sq = 1- (self.n-1)*(SSE) / ((self.n-k-1)*(SST))
    
        return ad_R_sq
    
    def weight(self) : 
        #chromomsome : 여러 염색체들이 모여진 C x m np.array. 
        result = []  
        for chrom in self.chromosome : 
            result.append(self.ad_R_sq(chrom))
        result = np.array(result) / sum(result) 
        return result
            

In [32]:
test = Genetic_Algorithm(X,y, model, "", 3, 0.01, 0.5)
test.create_chrom()
test.ad_R_sq([0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])
test.weight()

array([0.34134656, 0.29524741, 0.36340603])

### 4. 우수 염색체 선택(Selection)

**구현해야하는 것** 
- Deterministic / Probability 방식에 따라 Chromosome 구현 방식 구분 
- Deterministic 방식이라면, 상위 몇%까지 weight 의 값이 큰 경우 선택할 것
- Probability 방식이라면, weight 비중에 맞춰서 랜덤하게 선발할 것. 

**필요한 것**
- way : Deterministic / Probability 여부 
- select_r : 상위 몇 %까지 선발할 것인지 
- c : chrom의 개수 


**함수의 형태**
- selection(self, select_r) 


In [71]:
class Genetic_Algorithm() :
    def __init__(self, X, y, model, way, c, r, c_ratio) :
        self.X = X
        self.y = y
        self.model = model
        self.way = way
        self.c = c
        self.r = r 
        self.c_ratio = c_ratio
        
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]

        self.all_vars = np.array(range(self.n))
        self.chromosome = self.create_chrom()

    def create_chrom(self) : 
        chromosome = []
        for i in range(self.c) : 
            chrom = np.random.choice([0, 1], size=(self.m), p=[1-self.c_ratio, self.c_ratio]) 
            chromosome.append(chrom)
        return np.array(chromosome)
            
        
    def ad_R_sq(self, chrom) : 
        # chrom : 단일 염색체. np.array의 형식으로 0,1의 값으로 이뤄져 있음.
        select_val = [c for c, v in zip(self.all_vars, chrom) if v == 1]
        k = len(select_val)
        select_X = np.take(self.X, select_val, axis=1)
        mean_y = np.mean(self.y)

        self.model.fit(select_X,self.y)
        pred_y = self.model.predict(select_X)

        SST = np.sum((self.y - mean_y)**2)
        SSE = np.sum((self.y-pred_y)**2) 
    
        ad_R_sq = 1- (self.n-1)*(SSE) / ((self.n-k-1)*(SST))
    
        return ad_R_sq
    
    def weight(self) : 
        #chromomsome : 여러 염색체들이 모여진 C x m np.array. 
        result = []  
        for chrom in self.chromosome : 
            result.append(self.ad_R_sq(chrom))
        result = np.array(result) / sum(result) 
        return result
            
    
    def selection(self, select_r) : 
        select_num = int(select_r*self.c)
        result = []
        
        if self.way == "Deterministic" : 
            return self.chromosome[self.weight().argsort()[::-1][:select_num]]
        if self.way == "Probability" :   
            index = np.random.choice(range(len(self.chromosome)), select_num, p=self.weight(), replace=False)
            for i in index : result.append(self.chromosome[i]) 
            return np.array(result)
        
        if self.way != "Deterministic" and self.way != "Probability" : 
            return print("Please choose 1)Deterministic or 2) Probability")
            
        

In [72]:
test = Genetic_Algorithm(X,y, model, "Probability", 10, 0.01, 0.5)
test.selection(0.7)

array([[0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0]])

### 5. 다음 세대 염색체 생성(Crossoveer & Mutation)

**구현해야하는 것** 
- Crossover : 선택한 염색체들 간 변수값을 교체함.    
> 2개 염색체 선택하여 2개의 자식을 만들 것. 2개씩 랜덤으로 산출. 총 염색체 수를 가능한 유지시키기 위해 n//2 개 선정
> 선정한 쌍끼리 Crossover 진행
    
- Mutation : 일정 확률로 변수의 값을 변경함
> 각 변수의 난수가 mutation ratio 보다 낮게 나오면 값을 반전시키기. 

**필요한 것**
- selection 된 염색체 집합
- 난수 
- mutation 비율


**함수의 형태**
- crossover(self, select_r) 
- mutation(self) 



In [92]:
class Genetic_Algorithm() :
    def __init__(self, X, y, model, way, c, r, c_ratio) :
        self.X = X
        self.y = y
        self.model = model
        self.way = way
        self.c = c
        self.r = r 
        self.c_ratio = c_ratio
        
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]

        self.all_vars = np.array(range(self.n))
        self.chromosome = self.create_chrom()

    def create_chrom(self) : 
        chromosome = []
        for i in range(self.c) : 
            chrom = np.random.choice([0, 1], size=(self.m), p=[1-self.c_ratio, self.c_ratio]) 
            chromosome.append(chrom)
        return np.array(chromosome)
            
        
    def ad_R_sq(self, chrom) : 
        # chrom : 단일 염색체. np.array의 형식으로 0,1의 값으로 이뤄져 있음.
        select_val = [c for c, v in zip(self.all_vars, chrom) if v == 1]
        k = len(select_val)
        select_X = np.take(self.X, select_val, axis=1)
        mean_y = np.mean(self.y)

        self.model.fit(select_X,self.y)
        pred_y = self.model.predict(select_X)

        SST = np.sum((self.y - mean_y)**2)
        SSE = np.sum((self.y-pred_y)**2) 
    
        ad_R_sq = 1- (self.n-1)*(SSE) / ((self.n-k-1)*(SST))
    
        return ad_R_sq
    
    def weight(self) : 
        #chromomsome : 여러 염색체들이 모여진 C x m np.array. 
        result = []  
        for chrom in self.chromosome : 
            result.append(self.ad_R_sq(chrom))
        result = np.array(result) / sum(result) 
        return result
            
    
    def selection(self, select_r) : 
        select_num = int(select_r*self.c)
        result = []
        
        if self.way == "Deterministic" : 
            return self.chromosome[self.weight().argsort()[::-1][:select_num]]
        if self.way == "Probability" :   
            index = np.random.choice(range(len(self.chromosome)), select_num, p=self.weight(), replace=False)
            for i in index : result.append(self.chromosome[i]) 
            return np.array(result)
        
        if self.way != "Deterministic" and self.way != "Probability" : 
            return print("Please choose 1)Deterministic or 2) Probability")
            
    def crossover(self, select_r) : 
        select_chrom = self.selection(select_r)
        cross_list = [] 
        for i in range(self.n//2) :
            first,second = np.random.choice(range(len(select_chrom)), 2, replace=False)
            first_parent, second_parent = select_chrom[first],select_chrom[second]
            np.random.seed(i)
            random = np.random.rand(self.m)
            
            first_child, second_child = [], []
            for j, num in enumerate(random) : 
                if num >= 0.5 :
                    first_child.append(first_parent[j])
                    second_child.append(second_parent[j])
                else : 
                    first_child.append(second_parent[j])
                    second_child.append(first_parent[j])
            cross_list.append(first_child)
            cross_list.append(second_child)
        return np.array(cross_list)
    
    def mutation(self, chromosome) : 
        for i in range(len(chromosome)) : 
            np.random.seed(i)
            random = np.random.rand(self.m) 
            
            for j, num in enumerate(random) : 
                if num < self.m :
                    chromosome[i,j] = 1-chromosome[i,j]
        return chromosome
        


In [93]:
test = Genetic_Algorithm(X,y, model, "Probability", 10, 0.01, 0.5)
a= test.crossover(0.6)
test.mutation(a)

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 0, 1],
       [0, 1, 0, ..., 1, 0, 0]])

### 6. 최종 변수 집합 선택

**구현해야하는 것** 
- 원하는 수준까지 도달했을 때까지 계속 세대를 반복하는 것 


In [103]:
class Genetic_Algorithm() :
    def __init__(self, X, y, model, way, c, r, c_ratio) :
        self.X = X
        self.y = y
        self.model = model
        self.way = way
        self.c = c
        self.r = r 
        self.c_ratio = c_ratio
        
        self.n = np.shape(X)[0]
        self.m = np.shape(X)[1]

        self.all_vars = np.array(range(self.n))
        self.chromosome = self.create_chrom()

    def create_chrom(self) : 
        chromosome = []
        for i in range(self.c) : 
            chrom = np.random.choice([0, 1], size=(self.m), p=[1-self.c_ratio, self.c_ratio]) 
            chromosome.append(chrom)
        return np.array(chromosome)
            
        
    def ad_R_sq(self, chrom) : 
        # chrom : 단일 염색체. np.array의 형식으로 0,1의 값으로 이뤄져 있음.
        select_val = [c for c, v in zip(self.all_vars, chrom) if v == 1]
        k = len(select_val)
        select_X = np.take(self.X, select_val, axis=1)
        mean_y = np.mean(self.y)

        self.model.fit(select_X,self.y)
        pred_y = self.model.predict(select_X)

        SST = np.sum((self.y - mean_y)**2)
        SSE = np.sum((self.y-pred_y)**2) 
    
        ad_R_sq = 1- (self.n-1)*(SSE) / ((self.n-k-1)*(SST))
    
        return ad_R_sq
    
    def weight(self) : 
        #chromomsome : 여러 염색체들이 모여진 C x m np.array. 
        result = []  
        for chrom in self.chromosome : 
            result.append(self.ad_R_sq(chrom))
        result = np.array(result) / sum(result) 
        return result
            
    
    def selection(self, select_r) : 
        select_num = int(select_r*self.c)
        result = []
        
        if self.way == "Deterministic" : 
            return self.chromosome[self.weight().argsort()[::-1][:select_num]]
        if self.way == "Probability" :   
            index = np.random.choice(range(len(self.chromosome)), select_num, p=self.weight(), replace=False)
            for i in index : result.append(self.chromosome[i]) 
            return np.array(result)
        
        if self.way != "Deterministic" and self.way != "Probability" : 
            return print("Please choose 1)Deterministic or 2) Probability")
            
    def crossover(self, select_r) : 
        select_chrom = self.selection(select_r)
        cross_list = [] 
        for i in range(self.n//2) :
            first,second = np.random.choice(range(len(select_chrom)), 2, replace=False)
            first_parent, second_parent = select_chrom[first],select_chrom[second]
            np.random.seed(i)
            random = np.random.rand(self.m)
            
            first_child, second_child = [], []
            for j, num in enumerate(random) : 
                if num >= 0.5 :
                    first_child.append(first_parent[j])
                    second_child.append(second_parent[j])
                else : 
                    first_child.append(second_parent[j])
                    second_child.append(first_parent[j])
            cross_list.append(first_child)
            cross_list.append(second_child)
        return np.array(cross_list)
    
    def mutation(self, chromosome) : 
        for i in range(len(chromosome)) : 
            np.random.seed(i)
            random = np.random.rand(self.m) 
            
            for j, num in enumerate(random) : 
                if num < self.m :
                    chromosome[i,j] = 1-chromosome[i,j]
        return chromosome
        
    def genetic_goal(self, select_r, purpose) : 
        while max(self.weight()) > purpose : 
            crossover = self.crossover(select_r)
            mutation = self.mutation(crossover)
            self.chromosome = mutation 
        
        index = self.weight().argmax()
        return [c for c,v in zip(self.all_vars, self.chromosome[index]) if v ==1]

In [104]:
test = Genetic_Algorithm(X,y, model, "Probability", 10, 0.01, 0.5)
test.genetic_goal(0.5, 0.6)

[0, 1, 2, 4, 5, 7, 8, 11, 12]